###ART1 Algorithm

In [ ]:
import numpy as np
import os
import pandas as pd
from tensorflow.keras.datasets import mnist
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import random

class ART1:
    def __init__(self, input_size, vigilance=0.8):
        self.input_size = input_size
        self.vigilance = vigilance
        self.reset_network()

    def reset_network(self):
        self.top_down_weights = []
        self.bottom_up_weights = []

    def _initialize_node(self, x):
        self.top_down_weights.append(x.copy())
        b = x / (0.5 + np.sum(x))
        self.bottom_up_weights.append(b)

    def _choice_function(self, x, b):
        return np.sum(b * x)

    def _match_function(self, x, t):
        return np.sum(np.minimum(x, t)) / np.sum(x)

    def fit(self, X):
        n_samples = X.shape[0]
        self.labels_ = -np.ones(n_samples, dtype=int)

        for i in range(n_samples):
            x = X[i]
            activated_nodes = list(range(len(self.top_down_weights)))

            while activated_nodes:
                choices = np.array([self._choice_function(x, self.bottom_up_weights[j]) for j in activated_nodes])
                j_star_idx = np.argmax(choices)
                j_star = activated_nodes[j_star_idx]

                if self._match_function(x, self.top_down_weights[j_star]) >= self.vigilance:
                    self.top_down_weights[j_star] = np.minimum(self.top_down_weights[j_star], x)
                    b = self.top_down_weights[j_star] / (0.5 + np.sum(self.top_down_weights[j_star]))
                    self.bottom_up_weights[j_star] = b
                    self.labels_[i] = j_star
                    break
                else:
                    activated_nodes.pop(j_star_idx)

            if self.labels_[i] == -1:
                self._initialize_node(x)
                self.labels_[i] = len(self.top_down_weights) - 1

    def predict(self, X):
        preds = []
        for x in X:
            activated_nodes = list(range(len(self.top_down_weights)))
            while activated_nodes:
                choices = np.array([self._choice_function(x, self.bottom_up_weights[j]) for j in activated_nodes])
                j_star_idx = np.argmax(choices)
                j_star = activated_nodes[j_star_idx]

                if self._match_function(x, self.top_down_weights[j_star]) >= self.vigilance:
                    preds.append(j_star)
                    break
                else:
                    activated_nodes.pop(j_star_idx)

            if not activated_nodes:
                preds.append(-1)
        return np.array(preds)

    def get_num_clusters(self):
        return len(self.top_down_weights)

# Helper functions
def load_and_binarize_mnist(threshold=128, num_samples=2500):
    (X_train, y_train), (X_test, y_test) = mnist.load_data()
    X = np.concatenate((X_train, X_test))
    y = np.concatenate((y_train, y_test))
    X = X.reshape((X.shape[0], -1))
    X = (X > threshold).astype(int)
    indices = np.random.choice(X.shape[0], num_samples, replace=False)
    return X[indices], y[indices]

def plot_confusion(y_true, y_pred, save_path):
    cm = confusion_matrix(y_true, y_pred, labels=np.unique(y_true))
    disp = ConfusionMatrixDisplay(confusion_matrix=cm)
    disp.plot(cmap=plt.cm.Blues)
    plt.title("Confusion Matrix")
    plt.savefig(save_path)
    plt.close()

def visualize_clusters(X, y_pred, save_path, num_clusters=10):
    unique_clusters = np.unique(y_pred)
    fig, axs = plt.subplots(len(unique_clusters), 10, figsize=(20, 2 * len(unique_clusters)))
    if len(unique_clusters) == 1:
        axs = np.expand_dims(axs, 0)
    for idx, cluster_id in enumerate(unique_clusters):
        cluster_imgs = X[y_pred == cluster_id]
        sample_imgs = cluster_imgs[:10]
        for i, img in enumerate(sample_imgs):
            axs[idx, i].imshow(img.reshape(28, 28), cmap="gray")
            axs[idx, i].axis('off')
        for i in range(len(sample_imgs), 10):
            axs[idx, i].axis('off')
    plt.suptitle("Cluster Visualizations")
    plt.savefig(save_path)
    plt.close()

# Main Experiment
vigilance_values = [0.5, 0.6, 0.7, 0.8]
results = []

X, y = load_and_binarize_mnist(num_samples=2500)

for vigilance in vigilance_values:
    print(f"Training ART1 with vigilance {vigilance}...")
    model = ART1(input_size=784, vigilance=vigilance)
    model.fit(X)
    y_pred = model.labels_

    output_dir = f"results/vigilance_{vigilance}"
    os.makedirs(output_dir, exist_ok=True)

    plot_confusion(y, y_pred, save_path=os.path.join(output_dir, "confusion_matrix.png"))
    visualize_clusters(X, y_pred, save_path=os.path.join(output_dir, "cluster_visualization.png"))

    num_clusters = model.get_num_clusters()
    results.append((vigilance, num_clusters))

# Save CSV
results_df = pd.DataFrame(results, columns=["Vigilance", "NumClusters"])
results_df.to_csv("results/cluster_results.csv", index=False)

# Plot vigilance vs num clusters
plt.figure(figsize=(8,6))
plt.plot(results_df["Vigilance"], results_df["NumClusters"], marker='o')
plt.xlabel("Vigilance")
plt.ylabel("Number of Clusters")
plt.title("Vigilance vs Number of Clusters")
plt.grid(True)
plt.savefig("results/vigilance_vs_clusters.png")
plt.close()

***Custom Test Case***

In [ ]:
import numpy as np

# 5 input vectors manually (binary vectors) 
X_test = np.array([
    [1, 1, 0, 0, 0, 0, 1],
    [0, 0, 1, 1, 1, 1, 0],
    [1, 0, 1, 1, 1, 1, 0],
    [0, 0, 0, 1, 1, 1, 0],
    [1, 1, 0, 1, 1, 1, 0]
])

class ART1:
    def __init__(self, input_size, vigilance=0.7):
        self.input_size = input_size
        self.vigilance = vigilance
        self.reset_network()

    def reset_network(self):
        self.top_down_weights = []
        self.bottom_up_weights = []

    def _initialize_node(self, x):
        self.top_down_weights.append(x.copy())
        b = x / (0.5 + np.sum(x))
        self.bottom_up_weights.append(b)

    def _choice_function(self, x, b):
        return np.sum(b * x)

    def _match_function(self, x, t):
        return np.sum(np.minimum(x, t)) / np.sum(x)

    def fit(self, X):
        n_samples = X.shape[0]
        self.labels_ = -np.ones(n_samples, dtype=int)

        for i in range(n_samples):
            x = X[i]
            activated_nodes = list(range(len(self.top_down_weights)))

            while activated_nodes:
                choices = np.array([self._choice_function(x, self.bottom_up_weights[j]) for j in activated_nodes])
                j_star_idx = np.argmax(choices)
                j_star = activated_nodes[j_star_idx]

                if self._match_function(x, self.top_down_weights[j_star]) >= self.vigilance:
                    self.top_down_weights[j_star] = np.minimum(self.top_down_weights[j_star], x)
                    b = self.top_down_weights[j_star] / (0.5 + np.sum(self.top_down_weights[j_star]))
                    self.bottom_up_weights[j_star] = b
                    self.labels_[i] = j_star
                    break
                else:
                    activated_nodes.pop(j_star_idx)

            if self.labels_[i] == -1:
                self._initialize_node(x)
                self.labels_[i] = len(self.top_down_weights) - 1

    def get_top_down_weights(self):
        return np.array(self.top_down_weights)

# ART1 model with 7 inputs and vigilance 0.7
model = ART1(input_size=8, vigilance=0.7)

# Training the model
model.fit(X_test)

# Getting the final top-down weights (cluster templates)
final_top_down_weights = model.get_top_down_weights()

# Printing the top-down weights
print("\nFinal Top-Down Weights (Cluster Templates):")
for idx, weights in enumerate(final_top_down_weights):
    print(f"Cluster {idx} weights: {weights}")

# Also printing the cluster assignments
print("\nCluster Assignments for Each Input:")
print(model.labels_)


Final Top-Down Weights (Cluster Templates):
Cluster 0 weights: [1 1 0 0 0 0 1]
Cluster 1 weights: [0 0 0 1 1 1 0]
Cluster 2 weights: [1 1 0 1 1 1 0]

Cluster Assignments for Each Input:
[0 1 1 1 2]
